# Praca domowa nr 2

### Wojciech Celej
---

## Przygotowanie środowiska

In [1]:
import sqlite3
import pandas as pd
import numpy as np
badges = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Badges.csv.gz",
    comment="#", compression="gzip")
comments = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Comments.csv.gz",
    comment="#", compression="gzip")
postlinks = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/PostLinks.csv.gz",
    comment="#", compression="gzip")
posts = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Posts.csv.gz",
    comment="#", compression="gzip")
tags = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Tags.csv.gz",
    comment="#", compression="gzip")
users = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Users.csv.gz",
    comment="#", compression="gzip")
votes = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Votes.csv.gz",
    comment="#", compression="gzip")
conn = sqlite3.connect("pd2.db")

In [2]:
badges.to_sql("badges", conn)
comments.to_sql("comments", conn)
postlinks.to_sql("postlinks", conn)
posts.to_sql("posts", conn)
tags.to_sql("tags", conn)
users.to_sql("users", conn)
votes.to_sql("votes", conn)

---
## 1. Zapytanie 

Wybierz 10 użytkowników, będących właścicielami postów będących pytaniami, których to posty mają w sumie najwięcej kliknięć "Ulubione". Dla każdego użytkownika wyświetl: nick, wiek, adres, łączną liczbę polubień wszystkich jego pytań oraz tytuł najbardziej popularnego pytania wraz z liczbą kliknięć "Ulubione".

In [3]:
pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


Użycie zmiennej `result1` jest wskazane, ponieważ pojawia się w dwóch miejscach, ponieważ chcezmy wyznaczyć dwa parametry (`sum` i `max`) na tym samym Series. Użycie funkcji `join` jest tutaj wskazane, ponieważ złącza ona dwa DataFrame po indeksach (nie trzeba podawać warunku złącznia jak w `merge`). Użycie `idxmax` zamiast `max` jest potrzebne, ponieważ potrzebujemy pełnych rekordów, a nie tylko wartości. 

In [4]:
result1 = posts.query("PostTypeId==1").groupby("OwnerUserId")
result2 = result1["FavoriteCount"].sum().rename("FavoriteTotal").to_frame().join(posts.reindex(result1["FavoriteCount"].idxmax()). \
                                                                                 loc[:, ["Title", "FavoriteCount", "OwnerUserId"]].set_index("OwnerUserId")).reset_index()
users.loc[:, ["DisplayName", "Age", "Location", "Id"]].merge(result2, left_on="Id", right_on="OwnerUserId", how="inner").drop(columns=["Id", "OwnerUserId"]). \
    sort_values("FavoriteTotal", ascending=False).rename(columns={"Title": "MostFavoriteQuestion", "FavoriteCount": "MostFavoriteQuestionLikes"}).reset_index(drop=True).head(10)


,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


---
## 2. Zapytanie

Wybierz 10 postów z największą liczbą pozytywnych odpowiedzi (`score > 0`, też postów). Dla każdego postu wyświetl: id, tytuł, liczbę pozytywnych odpowiedzi.

In [5]:
pd.read_sql_query("""
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
    SELECT
        Posts.ParentID,
        COUNT(*) AS PositiveAnswerCount
    FROM Posts
    WHERE Posts.PostTypeID=2 AND Posts.Score>0
    GROUP BY Posts.ParentID
    ) AS Posts2
    ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
""", conn)

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


In [6]:
posts2 = posts.query("PostTypeId==2 and Score>0").groupby("ParentId").size().rename("PositiveAnswerCount").to_frame()
posts.loc[:, ["Id", "Title"]].merge(posts2, left_on="Id", right_on="ParentId", how="inner").sort_values("PositiveAnswerCount", ascending=False).reset_index(drop=True).head(10)

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,7663,Am I expected to tip wait staff in Europe?,17
9,59128,"Is there a way to prevent ""looking like a tour...",17


---
## 3. Zapytanie

Dla każdego roku wyświetl tytuł posta będącego pytaniem z największą liczbą głosów "Up" oraz liczbę tych głosów.

In [7]:
pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
    SELECT
        PostId,
        COUNT(*) AS Count,
        STRFTIME('%Y', Votes.CreationDate) AS Year
    FROM Votes
    WHERE VoteTypeId=2
    GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
""", conn)

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


Do stawienia nowego Series do DataFrame używam funkcji `insert`. Zwykłe wstawienie postaci `UpVotesPerYear["Year"] = ...` wyrzuca warninga.

In [8]:
UpVotesPerYear = votes.query("VoteTypeId==2")
UpVotesPerYear.insert(UpVotesPerYear.shape[1], "Year", pd.to_datetime(UpVotesPerYear.loc[:, "CreationDate"]).apply(lambda x: x.year))
result = posts.query("PostTypeId==1").loc[:, ["Title", "Id"]].merge(UpVotesPerYear.groupby(["PostId", "Year"]).size().rename("Count").reset_index(), left_on="Id", right_on="PostId", how="inner")
result.reindex(result.groupby("Year")["Count"].idxmax()).loc[:, ["Title", "Year", "Count"]].reset_index(drop=True)

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


---
## 4. Zapytanie

Wyświetl posty będące pytaniami, dla których różnica między oceną najwyżej ocenionej odpowiedzi, a oceną zaakceptowanej odpowiedzi jest większa od 50, posortuj malejąco względem tej różnicy. Dla każdego pytania podaj: id, tytuł, maksymalną ocenę odpowiedzi do tego pytania, ocenę zaakceptowanej odpowiedzi to tego pytania oraz różnicę tych wartości.

In [9]:
pd.read_sql_query("""
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
    SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
    SELECT * FROM Posts
    WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
""", conn)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


In [10]:
BestAnswers = posts.reindex(posts.query("PostTypeId==2").groupby("ParentId")["Score"].idxmax()).loc[:, ["ParentId", "Score"]]. \
    rename(columns={"Score": "MaxScore"})
Questions = posts.query("PostTypeId==1")
result1 = Questions.merge(BestAnswers, left_on="Id", right_on="ParentId", how="inner")
result2 = result1.merge(posts, left_on="AcceptedAnswerId", right_on="Id", how="inner").rename( \
    columns={"Score_y": "AcceptedScore", "Id_x": "Id", "Title_x": "Title"}).loc[:, ["Id", "Title", "MaxScore", "AcceptedScore"]]
result2.loc[:, "Difference"] = result2.loc[:, "MaxScore"] - result2.loc[:, "AcceptedScore"]
result2.query("Difference>50").sort_values("Difference", ascending=False).reset_index(drop=True)


,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


---
## 5. Zapytanie 

Wyświetl 10 postów będących pytaniami, do których komentarze twórców tych postów uzyskały łącznie najwyższą ocenę. Wyświetl tytuły tych pytań oraz łączną sumę ocen komentarzy twórców tych pytań.

In [11]:
pd.read_sql_query("""
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
    SELECT
        PostID,
        UserID,
        SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostID, UserID
    ) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""", conn)

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


In [12]:
CmtTotScr = comments.groupby(["PostId", "UserId"])["Score"].sum().rename("CommentsTotalScore").reset_index()
posts.query("PostTypeId==1").merge(CmtTotScr, left_on=["Id", "OwnerUserId"], right_on=["PostId", "UserId"], how="inner").loc[:, ["Title", "CommentsTotalScore"]]. \
        sort_values("CommentsTotalScore", ascending=False).reset_index(drop=True).head(10)

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


---
## 6. Zapytanie

Wyświetla dane(id, imię, reputację, wiek i lokalizację) użytkowników którzy mają od 2 do 10 złotych odznak.

In [13]:
pd.read_sql_query("""
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
    SELECT
        Name, UserID
    FROM Badges
    WHERE Name IN (
        SELECT
            Name
        FROM Badges
        WHERE Class=1
        GROUP BY Name
        HAVING COUNT(*) BETWEEN 2 AND 10
    )
    AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
""", conn)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,None
6,6669,Relaxed,69405.0,NaN,None
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


In [14]:
result1 = badges.query("Class==1").groupby("Name").size().rename("Count").to_frame().query("Count>1 and Count <11").reset_index()["Name"].astype(str)
ValuableBadges = badges.query("Class==1").loc[badges.loc[:, "Name"].isin(result1)].loc[:, ["Name", "UserId"]]
ValuableBadges.merge(users, left_on="UserId", right_on="Id").loc[:, ["Id", "DisplayName", "Reputation", "Age", "Location"]].drop_duplicates().reset_index(drop=True)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,NaN
6,6669,Relaxed,69405.0,NaN,NaN
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


---
## 7. Zapytanie

Wybierz tytuły 10 postów będących pytaniami z największą łączną ilością głosów z lat do 2015 roku typu "UpMod". Wyświetl tytuły postów i liczbę tych głosów.

In [15]:
pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
            WHEN '2017' THEN 'new'
            WHEN '2016' THEN 'new'
            ELSE 'old'
            END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, VoteDate
    ) AS VotesByAge
    GROUP BY VotesByAge.PostId
    HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
""", conn)

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


Wstawiam do `VotesByAge` Series klasyfikujący głos jako stary albo nowy. Następnie wstawiam dwa Series odpowiednio `NewVotes` i `OldVotes` wypełnione domyślnie zerami. 
Później wstawiane są odpowiednio wartości zgodnie z klasyfikacją stare/nowe. Podobnie jak w zapytaniu 1. warto zapamiętać result1 i następnie połączyć `join`.

In [16]:
VotesByAge = votes.query("VoteTypeId==2").loc[:, ["PostId", "CreationDate"]]
VotesByAge.loc[:, "Year"] = pd.to_datetime(VotesByAge.loc[:, "CreationDate"]).apply(lambda x: x.year)
VotesByAge.insert(VotesByAge.shape[1], "VoteDate", "old")
VotesByAge.loc[VotesByAge["Year"].isin([2016, 2017]), "VoteDate"] = "new"
VotesByAge = VotesByAge.groupby(["PostId", "VoteDate"]).size().rename("Total").reset_index()
VotesByAge.insert(VotesByAge.shape[1], "NewVotes", 0)
VotesByAge.insert(VotesByAge.shape[1], "OldVotes", 0)
VotesByAge.loc[VotesByAge["VoteDate"]=="new", "NewVotes"] = VotesByAge.loc[:, "Total"]
VotesByAge.loc[VotesByAge["VoteDate"]=="old", "OldVotes"] = VotesByAge.loc[:, "Total"]
result1 = VotesByAge.groupby("PostId")
VotesByAge2 = result1["Total"].sum().rename("Votes").to_frame().join(result1["NewVotes"].max().to_frame())
VotesByAge2 = VotesByAge2.join(result1["OldVotes"].max().to_frame()).query("NewVotes==0").reset_index()
posts.query("PostTypeId==1").merge(VotesByAge2, left_on="Id", right_on="PostId", how="inner").loc[:, ["Title", "OldVotes"]].sort_values("OldVotes", ascending=False).reset_index(drop=True).head(10)

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,Should I avoid overnight flights from Dubai to...,23
7,"Can a visitor who is ""of age"" in their country...",23
8,What is the business model of commercial free ...,23
9,Carrying medicines internationally for a friend,23


---